In [90]:
! pip install bertopic



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [91]:

from bertopic import BERTopic
import pandas as pd



In [92]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [93]:
import pandas as pd
import re
import gensim
from nltk.stem import WordNetLemmatizer

punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'         # define a string of punctuation symbols

# Functions to clean tweets
def remove_links(tweet):
    """Takes a string and removes web links from it"""
    tweet = re.sub(r'http\S+', '', tweet)   # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # remove bitly links
    tweet = tweet.strip('[link]')   # remove [links]
    tweet = re.sub(r'pic.twitter\S+','', tweet)
    return tweet

def remove_users(tweet):
    """Takes a string and removes retweet and @user information"""
   # tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@)', '', tweet)  # remove tweeted at
    return tweet

def remove_hashtags(tweet):
    """Takes a string and removes any hash tags"""
    tweet = re.sub('(#)', ' ', tweet)  # remove hash tags
    return tweet

def remove_av(tweet):
    """Takes a string and removes AUDIO/VIDEO tags or labels"""
    tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    return tweet

def tokenize(tweet):
    """Returns tokenized representation of words in lemma form excluding stopwords"""
    result = []
    for token in gensim.utils.simple_preprocess(tweet):
        if token not in gensim.parsing.preprocessing.STOPWORDS \
                and len(token) > 2:  # drops words with less than 3 characters
            result.append(lemmatize(token))
    return result

def lemmatize(token):
    """Returns lemmatization of a token"""
    return WordNetLemmatizer().lemmatize(token, pos='v')

def preprocess_tweet(tweet):
    """Main master function to clean tweets, stripping noisy characters, and tokenizing use lemmatization"""
    #tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    #tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
  # tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet_token_list = tokenize(tweet)  # apply lemmatization and tokenization
    tweet = ' '.join(tweet_token_list)
    tweet = re.sub(' amp ', '&', tweet)
    return tweet

def basic_clean(tweet):
    """Main master function to clean tweets only without tokenization or removal of stopwords"""
    # tweet = remove_users(tweet)
    # tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet = re.sub(' & ', '', tweet)
    return tweet

def tokenize_tweets(df):
    df['tokens'] = df.tweet.apply(preprocess_tweet)
    num_tweets = len(df)
    print('Complete. Number of Tweets that have been cleaned and tokenized : {}'.format(num_tweets))
    return df

In [94]:
df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/df_nlp_com1.csv",  lineterminator='\n')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning:

Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.



### Topic Modelling sui tweets delle prime cinque comunità per numero di users 

In [61]:
df1_com0 = df1[df1["community"]=="community_0"]
df1_com1 = df1[df1["community"]=="community_1"]
df1_com2 = df1[df1["community"]=="community_2"]
df1_com3 = df1[df1["community"]=="community_3"]
df1_com4 = df1[df1["community"]=="community_4"]


In [62]:
joinDf = [df1_com0, df1_com1, df1_com2, df1_com3, df1_com4]

In [63]:
df = pd.concat(joinDf)

In [64]:
df = df.reset_index(drop=True)

In [65]:
df

,id,conversation_id,created_at,date,time,timezone,user_id,name,place,tweet,...,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,community,original_community_extension
0,1218684471211245568,1218684471211245568,2020-01-18 23:59:54+00:00,2020-01-18,23:59:54,100,90497917,David Schor (Biden=#BlueTrump) 💚,NaN,#JoeBiden accusing the Sanders campaign of dis...,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,community_0,348
1,1242962294302105600,1242962294302105600,2020-03-25 23:51:18+00:00,2020-03-25,23:51:18,100,90497917,David Schor (Biden=#BlueTrump) 💚,NaN,#Bernie is the #AntiTrump #Biden is the #BlueT...,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,community_0,348
2,1217165029440552961,1217156937193807872,2020-01-14 19:22:11+00:00,2020-01-14,19:22:11,100,90497917,David Schor (Biden=#BlueTrump) 💚,NaN,@Rosario10i @BethLynch2020 No need to attack h...,...,NaN,NaN,NaN,"[{'screen_name': 'BethLynch2020', 'name': 'eh'...",NaN,NaN,NaN,NaN,community_0,348
3,1249384363738796032,1249005745582018560,2020-04-12 18:10:19+01:00,2020-04-12,18:10:19,100,90497917,David Schor (Biden=#BlueTrump) 💚,NaN,@AndyOstroy @JoeBiden Hey #Biden supporters: t...,...,NaN,NaN,NaN,"[{'screen_name': 'AndyOstroy', 'name': 'Andy O...",NaN,NaN,NaN,NaN,community_0,348
4,1218651141396946946,1218645723476111360,2020-01-18 21:47:28+00:00,2020-01-18,21:47:28,100,329832303,Kathy,NaN,@Bern2Bern @davidsirota @cspan @JoeBiden We ne...,...,NaN,NaN,NaN,"[{'screen_name': 'Bern2Bern', 'name': 'Bern', ...",NaN,NaN,NaN,NaN,community_0,348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3068,1262819077749460995,1262548526094929920,2020-05-19 19:55:04+01:00,2020-05-19,19:55:04,100,629963553,Lazyzimms that’s me,NaN,@CactusFlower_1 @Rionach_KAG @JoeBiden @POTUS ...,...,NaN,NaN,NaN,"[{'screen_name': 'CactusFlower_1', 'name': 'Ca...",NaN,NaN,NaN,NaN,community_4,61
3069,1262818979296526338,1262801067919597568,2020-05-19 19:54:41+01:00,2020-05-19,19:54:41,100,629963553,Lazyzimms that’s me,NaN,@CaliNeedsHelp @brkrjoe @EMastrro @GGlocksX @R...,...,NaN,NaN,NaN,"[{'screen_name': 'calineedshelp', 'name': 'cal...",NaN,NaN,NaN,NaN,community_4,61
3070,1262818811646029824,1262738465550217216,2020-05-19 19:54:01+01:00,2020-05-19,19:54:01,100,629963553,Lazyzimms that’s me,NaN,@magaxxoo @AdaraCensored @PatriotM1A777 @Senat...,...,NaN,NaN,NaN,"[{'screen_name': 'magaxxoo', 'name': '🇺🇸 𝕄𝔸𝔾𝔸𝕝...",NaN,NaN,NaN,NaN,community_4,61
3071,1262818688996184065,1262574734681821184,2020-05-19 19:53:32+01:00,2020-05-19,19:53:32,100,629963553,Lazyzimms that’s me,NaN,@kravrex @LakeMonsterCL @Italia191 @_HopeRises...,...,NaN,NaN,NaN,"[{'screen_name': 'Kravrex', 'name': 'Playball'...",NaN,NaN,NaN,NaN,community_4,61


In [66]:
df = df.drop_duplicates()

In [67]:
df["tweet"] = df.tweet.astype("str")

In [68]:
import nltk


In [69]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [70]:
df["preprocessed_tweet"] = df["tweet"].apply(preprocess_tweet)


In [71]:
df.value_counts()

Series([], dtype: int64)

In [72]:
df.community.value_counts()

community_0    1637
community_1     827
community_2     561
community_3      39
community_4       9
Name: community, dtype: int64

In [73]:
tweets = df.preprocessed_tweet.to_list()
timestamp = df.date.to_list()

In [74]:
topic_model = BERTopic(language="english", nr_topics = 30)
topics, probs = topic_model.fit_transform(tweets)

In [75]:
topic_model.get_topic_info()


,Topic,Count,Name
0,-1,1322,-1_biden_joebiden_trump_vote
1,0,522,0_tarareade_dueprocess_joebiden_titleix
2,1,131,1_covid_coronavirus_pandemic_trump
3,2,123,2_biden_joe_joebiden_bide
4,3,95,3_joebiden_nowthisnews_joebide_live
5,4,67,4_realdonaldtrump_joebide_joebiden_america
6,5,61,5_barackobama_obama_michelleobama_barack
7,6,49,6_petition_sign_story_write
8,7,47,7_realdonaldtrump_biden_bide_november
9,8,46,8_vote_ballot_joebiden_november


In [77]:
topic_model.visualize_barchart()


In [78]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamp, nr_bins=20)


In [79]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)


In [80]:
 topic_model.get_representative_docs(0)


['pepperoceanna lauraldunnesq repanniekuster ncvli nhcadsv amanda tokyoprouts senatorshaheen senatorhassan&look good joebiden allegedly say junior staffer tara reade digital penetration later say emily doe chanel miller brock turner get life time registration physical crime',
 'marczak rob sicken journalists play accusations joebiden go town lynch students like owenlabrie washington post start media mob&brockturner nytimes students presidential candidates',
 'seanspicer potus realdonaldtrump joebiden media need explain give joebiden gentle treatment directives come obama biden administration journalists immediately victim perpetrator oppose claimant&accuse']

### Topic Modelling su tutti i tweet del primo semestre

In [81]:
df1 = df1.drop_duplicates()

In [82]:
df1["tweet"] = df1.tweet.astype("str")

In [83]:
df1["preprocessed_tweet"] = df1["tweet"].apply(preprocess_tweet)


In [84]:
tweets = df1.preprocessed_tweet.to_list()
timestamp = df1.date.to_list()

In [85]:
topic_model = BERTopic(language="english", nr_topics = 30)
topics, probs = topic_model.fit_transform(tweets)

In [86]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,24809,-1_biden_joebiden_trump_vote
1,0,981,0_cannabis_jimcrow_food_ruleoflaw
2,1,940,1_msnbc_cnn_news_joebiden
3,2,883,2_black_racist_racism_ados
4,3,702,3_realdonaldtrump_bide_biden_president
5,4,690,4_tarareade_metoo_assault_sexual
6,5,685,5_teamjoe_joebiden_biden_joe
7,6,669,6_realdonaldtrump_unstable_goodness_mentally
8,7,667,7_coronavirus_covid_barackobama_virus
9,8,640,8_medicareforall_healthcare_medicare_aca


In [87]:
topic_model.visualize_barchart()

In [88]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamp, nr_bins=20)

In [89]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

### Topic modelling sulle comunità con più tweets (meno users)


In [119]:
df1['community'].value_counts()

community_46      1854
community_0       1637
community_83       997
community_1616     850
community_1        827
                  ... 
community_1286       1
community_1400       1
community_1337       1
community_1715       1
community_1247       1
Name: community, Length: 1863, dtype: int64

In [96]:
df1_com0 = df1[df1["community"]=="community_0"]
df1_com1 = df1[df1["community"]=="community_1"]
df1_com46 = df1[df1["community"]=="community_46"]
df1_com83 = df1[df1["community"]=="community_83"]
df1_com1616 = df1[df1["community"]=="community_1616"]



In [97]:
joinDf = [df1_com0, df1_com1, df1_com46, df1_com83, df1_com1616]

In [98]:
df = pd.concat(joinDf)

In [99]:
df = df.reset_index(drop=True)

In [100]:
df = df.drop_duplicates()

In [101]:
df["tweet"] = df.tweet.astype("str")

In [102]:
df["preprocessed_tweet"] = df["tweet"].apply(preprocess_tweet)

In [103]:
df.community.value_counts()

community_46      1854
community_0       1637
community_83       997
community_1616     850
community_1        827
Name: community, dtype: int64

In [104]:
tweets = df.preprocessed_tweet.to_list()
timestamp = df.date.to_list()



In [105]:
topic_model = BERTopic(language="english", nr_topics = 30)
topics, probs = topic_model.fit_transform(tweets)

In [106]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,2504,-1_joebiden_biden_trump_cannabis
1,0,312,0_metoo_women_allegations_tarareade
2,1,286,1_realdonaldtrump_biden_trump_november
3,2,265,2_biden_thank_teamjoe_teambiden
4,3,236,3_healthcare_medicareforall_healthinsurance_be...
5,4,197,4_voteblue_latinos_seaofblue_votejoe
6,5,173,5_presidency_billionaires_tell_bernie
7,6,156,6_cannabis_jimcrow_food_marijuana
8,7,148,7_racist_racistjoe_racism_trumplite
9,8,135,8_joebiden_joebide_time_air


In [107]:
topic_model.visualize_barchart()

In [108]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamp, nr_bins=20)

In [109]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)